In [44]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import random
from scipy import sparse as sps
import importlib

In [45]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [46]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)
tgt_playlists.shape

(4420, 1)

In [47]:
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album', 'playcount'],idf=True, n_min_attr=90, n_el_sim=66)
tsr.fit(tracks)
print('Model fitted!')

Fixed dataset
Calculated Indices


  0%|          | 19/100000 [00:00<09:12, 181.12it/s]

ICM built
ICM regularized with IDF!


100%|██████████| 100000/100000 [24:06<00:00, 69.13it/s]

Similarity built
Model fitted!


In [48]:
_, IX_tgt_items, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:06<00:00, 642.54it/s]


URM built


In [49]:
print(URM.shape)
print(tsr.S.shape)
URM

(4420, 100000)
(100000, 100000)


<4420x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 165758 stored elements in Compressed Sparse Row format>

In [50]:
est_URM = URM.dot(tsr.S)

In [51]:
est_URM

<4420x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 3077092 stored elements in Compressed Sparse Row format>

In [52]:
IBR= importlib.reload(IBR)
ibr = IBR.ItemBasedRecommender(shrinkage=20,n_el_sim=65)
ibr.fit(tracks,train,tgt_tracks, URM=est_URM)
print('Model fitted!')

  0%|          | 12/100000 [00:00<14:02, 118.72it/s]

Calculated Indices


100%|██████████| 100000/100000 [37:39<00:00, 44.25it/s]


Similarity built
(100000, 17120)
Model fitted!


In [ ]:
print(URM.shape)
print(ibr.S.shape)
IX_tgt_items

(4420, 100000)
(100000, 17120)


In [1]:
rs = importlib.reload(rs)
recommendetions = np.array([])
for p in IX_tgt_playlists.values:
    #if normalize:
    #   tsr_avg_sims = np.divide(URM[p,:].dot(TopSimilarRecommender.S).toarray(), norm_dividend).ravel()
    avg_sims = URM[p,:].dot(ibr.S).toarray().ravel()
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, ibr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_df =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

NameError: name 'importlib' is not defined

In [ ]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [ ]:
map_eval = evaluate(rec_df, test, 'MAP')
print('Evaluation completed!')

In [ ]:
map_eval